<a href="https://colab.research.google.com/github/vuhpham94/nflx-data-projects-g6/blob/AL/project_final/notebooks/data_collection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# gdrive connection
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
import cv2     # for capturing videos
import math   # for mathematical operations
import matplotlib.pyplot as plt    # for plotting the images
%matplotlib inline
import pandas as pd
from keras.preprocessing import image   # for preprocessing the images
import numpy as np    # for mathematical operations
from keras.utils import np_utils
from skimage.transform import resize   # for resizing images
from sklearn.model_selection import train_test_split
from glob import glob
from tqdm import tqdm

In [2]:
# open the .txt file which have names of training videos
train_path = "/content/drive/MyDrive/ucfTrainTestlist/trainlist01.txt"
f = open(train_path, "r")
temp = f.read()
videos = temp.split('\n')
# creating a dataframe having video names
train = pd.DataFrame()
train['video_name'] = videos
train = train[:-1]
train.head()

,video_name
0,ApplyEyeMakeup/v_ApplyEyeMakeup_g08_c01.avi 1
1,ApplyEyeMakeup/v_ApplyEyeMakeup_g08_c02.avi 1
2,ApplyEyeMakeup/v_ApplyEyeMakeup_g08_c03.avi 1
3,ApplyEyeMakeup/v_ApplyEyeMakeup_g08_c04.avi 1
4,ApplyEyeMakeup/v_ApplyEyeMakeup_g08_c05.avi 1


In [ ]:
# open the .txt file which have names of test videos
test_path = "/content/drive/MyDrive/ucfTrainTestlist/testlist01.txt"
f = open(test_path, "r")
temp = f.read()
videos = temp.split('\n')
# creating a dataframe having video names
test = pd.DataFrame()
test['video_name'] = videos
test = test[:-1]
test.head()

In [4]:
# creating tags for training videos
train_video_tag = []
test_debug = []
for i in range(train.shape[0]):
  train_video_tag.append(train['video_name'][i].split('/')[0])
  
train['tag'] = train_video_tag
train.head()


,video_name,tag
0,ApplyEyeMakeup/v_ApplyEyeMakeup_g08_c01.avi 1,ApplyEyeMakeup
1,ApplyEyeMakeup/v_ApplyEyeMakeup_g08_c02.avi 1,ApplyEyeMakeup
2,ApplyEyeMakeup/v_ApplyEyeMakeup_g08_c03.avi 1,ApplyEyeMakeup
3,ApplyEyeMakeup/v_ApplyEyeMakeup_g08_c04.avi 1,ApplyEyeMakeup
4,ApplyEyeMakeup/v_ApplyEyeMakeup_g08_c05.avi 1,ApplyEyeMakeup


In [5]:

# creating tags for test videos
test_video_tag = []
for i in range(test.shape[0]):
  test_video_tag.append(test['video_name'][i].split('/')[0])
test['tag'] = test_video_tag
test.head()

,video_name,tag
0,ApplyEyeMakeup/v_ApplyEyeMakeup_g01_c01.avi,ApplyEyeMakeup
1,ApplyEyeMakeup/v_ApplyEyeMakeup_g01_c02.avi,ApplyEyeMakeup
2,ApplyEyeMakeup/v_ApplyEyeMakeup_g01_c03.avi,ApplyEyeMakeup
3,ApplyEyeMakeup/v_ApplyEyeMakeup_g01_c04.avi,ApplyEyeMakeup
4,ApplyEyeMakeup/v_ApplyEyeMakeup_g01_c05.avi,ApplyEyeMakeup


In [33]:
# storing the frames from training videos
import pdb; 

test_check = []
for i in tqdm(range(train.shape[0])):
  count = 0
  videoFile = train['video_name'][i]
  tag = train['tag'][i]
  #print(videoFile)
  #cap = cv2.VideoCapture('/content/drive/MyDrive/UCF-101/'+videoFile.split(' ')[0].split('/')[1])
  # capturing the video from the given path
  #print(tag)
  #print(videoFile.split(' ')[0].split('/')[1])
  cap = cv2.VideoCapture('/content/drive/MyDrive/UCF-101/'+tag+'/'+videoFile.split(' ')[0].split('/')[1]) 
  #pdb.set_trace()
  frameRate = cap.get(5) #frame rate
  x=1
  #pdb.set_trace()
  while(cap.isOpened()):
    frameId = cap.get(1) #current frame number
    ret, frame = cap.read()
    if (ret != True):
      break
    #pdb.set_trace()
    if (frameId % math.floor(frameRate) == 0):
      #storing the frames in a new folder named train_1
      filename ='train_1/' + videoFile.split('/')[1].split(' ')[0] +"_frame%d.jpg" % count;
      count+=1
      cv2.imwrite(filename, frame)
  cap.release()


100%|██████████| 9537/9537 [45:21<00:00,  3.50it/s]


In [34]:
# getting the names of all the images
images = glob("train_1/*.jpg")
train_image = []
train_class = []
for i in tqdm(range(len(images))):
# creating the image name
  train_image.append(images[i].split('/')[1])
# creating the class of image
  train_class.append(images[i].split('/')[1].split('_')[1])
# storing the images and their class in a dataframe
train_data = pd.DataFrame()
train_data['image'] = train_image
train_data['class'] = train_class
# converting the dataframe into csv file 
train_data.to_csv('train_new.csv',header=True, index=False)

100%|██████████| 65012/65012 [00:00<00:00, 538193.60it/s]


In [35]:
import keras
from keras.models import Sequential
from keras.applications.vgg16 import VGG16
from keras.layers import Dense, InputLayer, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, GlobalMaxPooling2D
from keras.preprocessing import image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.model_selection import train_test_split

In [36]:
train = pd.read_csv('train_new.csv')
train.head()

,image,class
0,v_CliffDiving_g17_c05.avi_frame3.jpg,CliffDiving
1,v_BandMarching_g17_c02.avi_frame3.jpg,BandMarching
2,v_Surfing_g08_c01.avi_frame7.jpg,Surfing
3,v_GolfSwing_g18_c06.avi_frame10.jpg,GolfSwing
4,v_Basketball_g12_c02.avi_frame0.jpg,Basketball


In [ ]:
# creating an empty list
train_image = []
# for loop to read and store frames
for i in tqdm(range(train.shape[0])):
# loading the image and keeping the target size as (224,224,3)
  img = image.load_img('train_1/'+train['image'][i], target_size=(224,224,3))
  # converting it to array
  img = image.img_to_array(img)
  # normalizing the pixel value
  img = img/255
  # appending the image to the train_image list
  train_image.append(img)
# converting the list to numpy array
X = np.array(train_image)
# shape of the array
X.shape

 30%|███       | 19644/65012 [01:25<05:09, 146.52it/s]

##Creating a Validation Set

In [ ]:
# separating the target
y = train['class']
# creating the training and validation set
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.2, stratify = y)

In [ ]:
# creating dummies of target variable for train and validation set
y_train = pd.get_dummies(y_train)
y_test = pd.get_dummies(y_test)

In [ ]:
# creating the base model of pre-trained VGG16 model
base_model = VGG16(weights='imagenet', include_top=False)

In [ ]:
# extracting features for training frames
X_train = base_model.predict(X_train)
X_train.shape

In [ ]:
# extracting features for validation frames
X_test = base_model.predict(X_test)
X_test.shape

In [ ]:
# reshaping the training as well as validation frames in single dimension
X_train = X_train.reshape(59075, 7*7*512)
X_test = X_test.reshape(14769, 7*7*512)

In [ ]:
# normalizing the pixel values
max = X_train.max()
X_train = X_train/max
X_test = X_test/max

In [ ]:
# shape of images
X_train.shape

In [ ]:
#defining the model architecture
model = Sequential()
model.add(Dense(1024, activation='relu', input_shape=(25088,)))
model.add(Dropout(0.5))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(101, activation='softmax'))

##Training the video classification model

In [ ]:
# defining a function to save the weights of best model
from keras.callbacks import ModelCheckpoint
mcp_save = ModelCheckpoint('weight.hdf5', save_best_only=True, monitor='val_loss', mode='min')

In [ ]:
# compiling the model
model.compile(loss='categorical_crossentropy',optimizer='Adam',metrics=['accuracy'])

In [ ]:
# training the model
model.fit(X_train, y_train, epochs=200, validation_data=(X_test, y_test), callbacks=[mcp_save], batch_size=128)

##Defining model architecture and loading weights

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.preprocessing import image
import numpy as np
import pandas as pd
from tqdm import tqdm
from keras.applications.vgg16 import VGG16
import cv2
import math
import os
from glob import glob
from scipy import stats as s

In [ ]:
base_model = VGG16(weights='imagenet', include_top=False)

In [ ]:
#defining the model architecture
model = Sequential()
model.add(Dense(1024, activation='relu', input_shape=(25088,)))
model.add(Dropout(0.5))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(101, activation='softmax'))

In [ ]:
# loading the trained weights
model.load_weights("weights.hdf5")

In [ ]:

# compiling the model
model.compile(loss='categorical_crossentropy',optimizer='Adam',metrics=['accuracy'])
view raw

##Creating the test data

In [ ]:
# getting the test list
f = open("testlist01.txt", "r")
temp = f.read()
videos = temp.split('\n')
# creating the dataframe
test = pd.DataFrame()
test['video_name'] = videos
test = test[:-1]
test_videos = test['video_name']
test.head()

In [ ]:
# creating the tags
train = pd.read_csv('UCF/train_new.csv')
y = train['class']
y = pd.get_dummies(y)


##Generating predictions for test videos

In [ ]:
# creating two lists to store predicted and actual tags
predict = []
actual = []
# for loop to extract frames from each test video
for i in tqdm(range(test_videos.shape[0])):
count = 0
videoFile = test_videos[i]
cap = cv2.VideoCapture('UCF/'+videoFile.split(' ')[0].split('/')[1])   # capturing the video from the given path
frameRate = cap.get(5) #frame rate
x=1
# removing all other files from the temp folder
files = glob('temp/*')
for f in files:
os.remove(f)
while(cap.isOpened()):
frameId = cap.get(1) #current frame number
ret, frame = cap.read()
if (ret != True):
break
if (frameId % math.floor(frameRate) == 0):
# storing the frames of this particular video in temp folder
filename ='temp/' + "_frame%d.jpg" % count;count+=1
cv2.imwrite(filename, frame)
cap.release()
# reading all the frames from temp folder
images = glob("temp/*.jpg")
prediction_images = []
for i in range(len(images)):
img = image.load_img(images[i], target_size=(224,224,3))
img = image.img_to_array(img)
img = img/255
prediction_images.append(img)
# converting all the frames for a test video into numpy array
prediction_images = np.array(prediction_images)
# extracting features using pre-trained model
prediction_images = base_model.predict(prediction_images)
# converting features in one dimensional array
prediction_images = prediction_images.reshape(prediction_images.shape[0], 7*7*512)
# predicting tags for each array
prediction = model.predict_classes(prediction_images)
# appending the mode of predictions in predict list to assign the tag to the video
predict.append(y.columns.values[s.mode(prediction)[0][0]])
# appending the actual tag of the video
actual.append(videoFile.split('/')[1].split('_')[1])

##Evaluating the model

In [ ]:
# checking the accuracy of the predicted tags
from sklearn.metrics import accuracy_score
accuracy_score(predict, actual)*100